<a href="https://colab.research.google.com/github/msolorzan/TensorFlow-projects/blob/main/04_Transfer_Learning_in_TensorFlow_Part_1_Feature_Extraction_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tranfer learning

Build and fit a model using the same data we have here but with the MobileNetV2 architecture feature extraction (mobilenet_v2_100_224/feature_vector) from TensorFlow Hub, how does it perform compared to our other models?

In [ ]:
!nvidia-smi

Sat Oct 29 22:00:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get data

In [ ]:
# Get data (10% of 10 food classes from Food101) - https://www.kaggle.com/dansbecker/food-101
# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

import zipfile

# Unzip the downloades file
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

--2022-10-29 22:00:54--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.128, 142.250.103.128, 108.177.120.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   211MB/s    in 0.8s    

2022-10-29 22:00:55 (211 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



## Inspect data

In [ ]:
import os

# How many images in each folder?
for dirpath, dirnames, filenames in os.walk('/content/10_food_classes_10_percent'):
  print(f'There are {len(dirnames)} directories and {len(filenames)} images in "{dirpath}"')

There are 2 directories and 0 images in "/content/10_food_classes_10_percent"
There are 10 directories and 0 images in "/content/10_food_classes_10_percent/train"
There are 0 directories and 75 images in "/content/10_food_classes_10_percent/train/fried_rice"
There are 0 directories and 75 images in "/content/10_food_classes_10_percent/train/chicken_curry"
There are 0 directories and 75 images in "/content/10_food_classes_10_percent/train/ice_cream"
There are 0 directories and 75 images in "/content/10_food_classes_10_percent/train/chicken_wings"
There are 0 directories and 75 images in "/content/10_food_classes_10_percent/train/sushi"
There are 0 directories and 75 images in "/content/10_food_classes_10_percent/train/grilled_salmon"
There are 0 directories and 75 images in "/content/10_food_classes_10_percent/train/ramen"
There are 0 directories and 75 images in "/content/10_food_classes_10_percent/train/pizza"
There are 0 directories and 75 images in "/content/10_food_classes_10_perce

### Creating train and test data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SHAPE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5

train_dir = '/content/10_food_classes_10_percent/train'
test_dir = '/content/10_food_classes_10_percent/test'

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

print('Training images')
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMG_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = 'categorical')
print('Testing images')
test_data = test_datagen.flow_from_directory(test_dir,
                                            target_size = IMG_SHAPE,
                                            batch_size = BATCH_SIZE,
                                            class_mode = 'categorical')

Training images
Found 750 images belonging to 10 classes.
Testing images
Found 2500 images belonging to 10 classes.


## Setting up callbacks

In [ ]:
import datetime
import tensorflow as tf

def create_tensorboard_callbacks(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%d/%m/%Y - %H:%M:%S')
  tensorboard_calback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f'Saved Tensorboard lof files in: {log_dir}')
  return tensorboard_calback

## Creating models using TensorFlow Hub

In the past we've used TensorFlow to create our own models layer by layer from scratch.

Now we're going to do a similar process, except the majroity of our model's layers are going to come from TensorFlow Hub.

We can access pretrained models on: https://tfhub.dev/

Browsing the TensorFlow Hub page and sorting for image classification, we found the following feature vector model link: https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5

In [ ]:
mobilenet_v2_100_224_url = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5'

In [ ]:
import tensorflow_hub as hub

def create_model(model_url, num_classes, image_shape):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

  Args:
  model_url (str): A TensorFlow Hub feature extraction URL.
  num_classes (int): Number of output neurons in the output layer,
  should be equal to number of target classes.

  Returns:
  An uncompiled Keras Sequential model with model_url as feature extractor
  layer and Dense output layer with num_classes output neurons.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5',
                                          trainable = False,
                                          name = 'feature_extraction_layer',
                                          input_shape = image_shape + (3,))
  
  # Create our own sequential model
  model = tf.keras.Sequential([
      feature_extractor_layer,
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ])

  return model

In [ ]:
mobilenet = create_model(mobilenet_v2_100_224_url, 
             train_data_10_percent.num_classes, 
             IMG_SHAPE)


In [ ]:
# Summarize the model
mobilenet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 1001)             3540265   
 erasLayer)                                                      
                                                                 
 dense_1 (Dense)             (None, 10)                10020     
                                                                 
Total params: 3,550,285
Trainable params: 10,020
Non-trainable params: 3,540,265
_________________________________________________________________


In [ ]:
# Compile the mobilenet
mobilenet.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
# Fit the model
mobilenet_history = mobilenet.fit(train_data_10_percent,
                                  epochs = EPOCHS,
                                  steps_per_epoch = len(train_data_10_percent),
                                  validation_data = test_data,
                                  validation_steps = len(test_data),
                                  callbacks = create_tensorboard_callbacks('tensorflow_hub', 'mobilenet_v2_100_224'))

Saved Tensorboard lof files in: tensorflow_hub/mobilenet_v2_100_224/29/10/2022 - 22:31:50
Epoch 1/5
24/24 [==============================] - 15s 641ms/step - loss: 0.5572 - accuracy: 0.8400 - val_loss: 0.6987 - val_accuracy: 0.7676
Epoch 2/5
24/24 [==============================] - 15s 643ms/step - loss: 0.4240 - accuracy: 0.8747 - val_loss: 0.6722 - val_accuracy: 0.7768
Epoch 3/5
24/24 [==============================] - 15s 642ms/step - loss: 0.3366 - accuracy: 0.9093 - val_loss: 0.6633 - val_accuracy: 0.7784
Epoch 4/5
24/24 [==============================] - 15s 639ms/step - loss: 0.2720 - accuracy: 0.9520 - val_loss: 0.6418 - val_accuracy: 0.7872
Epoch 5/5
24/24 [==============================] - 15s 636ms/step - loss: 0.2240 - accuracy: 0.9680 - val_loss: 0.6368 - val_accuracy: 0.7876
